Life‐Course Migration Indicators
Variables:
1. Childhood Migration:
DM017: Place of birth
DM019: “Where have you lived for most of your childhood (up to age 14)?”
2. Adult Migration:
DM017: Place of birth
DM020: “Where have you lived for most of your adult life?”
Coding:
1. Childhood Migration Indicator:
If DM017 differs from DM019, assign 1 (indicating a childhood migration); otherwise, 0.
2. Adult Migration Indicator:
If DM017 differs from DM020, assign 1 (indicating an adult migration); otherwise, 0.
Rationale: These indicators help differentiate migration experiences at different stages of life.


Input Variables:

DM017 ("Where is your place of birth?") and
DM018 ("Where were you living before coming to this place?")
are used to extract the rural/urban codes.
Classification:
The function classify_migration compares the codes:

1 for village (rural)
2 for town (urban)
and returns a string label indicating the type of migration.
Handling Missing Values:
If either variable is missing (or not 1 or 2), the function returns np.nan (you can customize this behavior if needed).

In [ ]:
# Get variable names by column labels (using your previously defined function)
col_DM017 = get_varname_by_label(meta_fp3, "Place of residence")
col_DM018 = get_varname_by_label(meta_fp3, "Place of last residence-village/town")

# Extract the columns for DM017 and DM018
# (These variables are assumed to contain numeric codes: 1 for village, 2 for town)
place_birth = df_fp3[col_DM017]
place_last_res = df_fp3[col_DM018]

# For clarity, create a new DataFrame with only the two variables:
df_migration_type = pd.DataFrame({
    "Place of Birth": place_birth,
    "Place of Last Residence": place_last_res
})

# Define a function to classify migration type based on rural/urban codes
def classify_migration(row):
    pb = row["Place of Birth"]
    plr = row["Place of Last Residence"]
    # Check for missing values (if not coded as 1 or 2, you might want to handle them separately)
    if pd.isna(pb) or pd.isna(plr):
        return np.nan
    if pb == 1 and plr == 1:
        return "Rural-to-Rural"
    elif pb == 1 and plr == 2:
        return "Rural-to-Urban"
    elif pb == 2 and plr == 1:
        return "Urban-to-Rural"
    elif pb == 2 and plr == 2:
        return "Urban-to-Urban"
    else:
        return np.nan

# Apply the function to each row to create a new column 'Migration Type'
df_migration_type["Migration Type"] = df_migration_type.apply(classify_migration, axis=1)

# Optionally, display the first few rows of the resulting DataFrame
print(df_migration_type.head())


In [ ]:

# Retrieve variable names for place of birth
col_birth_country = get_varname_by_label(meta_fp3, "Place of birth-country")
col_birth_state   = get_varname_by_label(meta_fp3, "Place of birth-state")
col_birth_district = get_varname_by_label(meta_fp3, "Place of birth-district")
col_birth_village  = get_varname_by_label(meta_fp3, "Place of birth-village/town")

# Retrieve variable names for childhood residence
col_child_country = get_varname_by_label(meta_fp3, "Lived most of your childhood-Country")
col_child_district = get_varname_by_label(meta_fp3, "Lived most of your childhood-distirct")
col_child_state   = get_varname_by_label(meta_fp3, "Lived most of your childhood-state")
col_child_village  = get_varname_by_label(meta_fp3, "Lived most of your childhood-village/town")

# Retrieve variable names for adult residence
col_adult_country = get_varname_by_label(meta_fp3, "Lived most of your adult life-country")
col_adult_state   = get_varname_by_label(meta_fp3, "Lived most of your adult life-state")
col_adult_district = get_varname_by_label(meta_fp3, "Lived most of your adult life-district")
col_adult_village  = get_varname_by_label(meta_fp3, "Lived most of your adult life-village/town")

# Check that all required variables were found
for name, col in [("Place of birth-country", col_birth_country),
                  ("Place of birth-state", col_birth_state),
                  ("Place of birth-district", col_birth_district),
                  ("Place of birth-village/town", col_birth_village),
                  ("Lived most of your childhood-Country", col_child_country),
                  ("Lived most of your childhood-distirct", col_child_district),
                  ("Lived most of your childhood-state", col_child_state),
                  ("Lived most of your childhood-village/town", col_child_village),
                  ("Lived most of your adult life-country", col_adult_country),
                  ("Lived most of your adult life-state", col_adult_state),
                  ("Lived most of your adult life-district", col_adult_district),
                  ("Lived most of your adult life-village/town", col_adult_village)]:
    if col is None:
        raise ValueError(f"Column label '{name}' not found in fp3.sav metadata.")

# Extract the relevant columns and clean the data
def clean_series(series):
    return series.astype(str).str.strip().str.lower().fillna("")

birth_country = clean_series(df_fp3[col_birth_country])
birth_state = clean_series(df_fp3[col_birth_state])
birth_district = clean_series(df_fp3[col_birth_district])
birth_village = clean_series(df_fp3[col_birth_village])

child_country = clean_series(df_fp3[col_child_country])
child_state = clean_series(df_fp3[col_child_state])
child_district = clean_series(df_fp3[col_child_district])
child_village = clean_series(df_fp3[col_child_village])

adult_country = clean_series(df_fp3[col_adult_country])
adult_state = clean_series(df_fp3[col_adult_state])
adult_district = clean_series(df_fp3[col_adult_district])
adult_village = clean_series(df_fp3[col_adult_village])

# Create concatenated location strings (using a separator to clearly mark boundaries)
birth_location = birth_country + "|" + birth_state + "|" + birth_district + "|" + birth_village
child_location = child_country + "|" + child_state + "|" + child_district + "|" + child_village
adult_location = adult_country + "|" + adult_state + "|" + adult_district + "|" + adult_village

# Compute migration indicators:
# Childhood Migration Indicator: 1 if any level differs between place of birth and childhood residence, else 0.
child_migration = (birth_location != child_location).astype(int)

# Adult Migration Indicator: 1 if any level differs between place of birth and adult residence, else 0.
adult_migration = (birth_location != adult_location).astype(int)

# Create a new DataFrame with the concatenated locations and migration indicators
df_life_course_migration = pd.DataFrame({
    "Birth Location": birth_location,
    "Childhood Location": child_location,
    "Childhood Migration Indicator": child_migration,
    "Adult Location": adult_location,
    "Adult Migration Indicator": adult_migration
})

# Display the first few rows of the resulting DataFrame
print(df_life_course_migration.head())
